LIBRERIE

In [ ]:
pip install nltk numpy

In [ ]:
pip install nltk

GRAMATICALLY / SYNTAX

Metrics

In [ ]:
%pip install evaluate

In [ ]:
%pip install jiwer

In [ ]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

READABILITY

Metrics

In [ ]:
pip install textstat

FACTUALITY/INFDENSITY

Metrics

In [ ]:
pip install sentence-transformers


In [ ]:
pip install transformers


In [ ]:
pip install bert_score

In [ ]:
# Factuality
def check_fact_quantified(claim, context, max_length=512):
    inputs = nli_tokenizer(
        context,
        claim,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
        padding=True
    )
    
    outputs = nli_model_raw(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1).squeeze()
    
    contradiction_index = nli_model_raw.config.label2id.get("CONTRADICTION", 2)  
    return probs[contradiction_index].item()

# Information Density
def information_density(text):
    doc = nlp(text)
    return len(doc.ents) / len(doc) if len(doc) > 0 else 0.0

#Information Relevance
def information_relevance(question, text):
    embeddings = sbert_model.encode([question, text])
    return util.cos_sim(embeddings[0], embeddings[1]).item()

COHERENCE

Metrics

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

#Local coherence
def compute_local_coherence(text):
    sentences = sent_tokenize(text)  
    if len(sentences) < 2:
        return 0.0

    # Compute Embeddings
    embeddings = embedding_model.encode(sentences, convert_to_tensor=True)

    # Similarity Matrix
    cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

    local_scores = [cosine_scores[i][i + 1].item() for i in range(len(sentences) - 1)]

    return round(np.mean(local_scores), 4)

SENTENCE FLOW

Metrics

In [ ]:
from scipy.stats import kendalltau

# Normalized order score (Kendall tau)
def normalized_order_score(original_sentences, modified_sentences):

    if len(original_sentences) != len(modified_sentences):
        raise ValueError("Different number of sentences")

    # Map each original sentence to its position
    position_map = {s: i for i, s in enumerate(original_sentences)}
    
    try:
        modified_indices = [position_map[s] for s in modified_sentences]
    except KeyError:
        raise ValueError("A modified sentence is not present in the original")

    # Compute Kendall tau correlation
    tau, _ = kendalltau(range(len(original_sentences)), modified_indices)
    print(tau)
    
    if tau is None:
        return 0.0  # maximum disorder in case of error

    # Normalize to [0,1], 1 = perfect order, 0 = reverse order
    return (tau + 1) / 2


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

#Semantic Flow
def semantic_coherence_flow(sentences):

    if len(sentences) < 2:
        return 1.0  

    embeddings = model.encode(sentences, convert_to_tensor=True)

    similarities = [
        util.cos_sim(embeddings[i], embeddings[i + 1]).item()
        for i in range(len(sentences) - 1)
    ]

    return sum(similarities) / len(similarities)
